In [1]:
import pandas as pd
import numpy as np

import time
import math
import random

from collections import defaultdict

#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split  

from sklearn.metrics import accuracy_score

In [32]:
raw_data = pd.read_csv('/Users/guozhiqi-seven/Documents/Statistical Learning/NaiveBayes/train.csv',header=0)

In [33]:
raw_data.shape

(42000, 785)

In [34]:
raw_data = raw_data[:10000]
data = raw_data.values
data.shape

(10000, 785)

In [35]:
data

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [4, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [36]:
imgs = data[:,1:]
labels = data[:,0] 

In [37]:
train_set, test_set, train_labels, test_labels = train_test_split(imgs,labels,test_size=0.33,random_state=23333)

In [38]:
train_set.shape

(6700, 784)

In [39]:
def rebuild_features(features):
    new_features=[]
    
    for feature in features:
        new_feature = []
        
        for i,j in enumerate(feature):
            new_feature.append(str(i)+'_'+str(j))
        
        new_features.append(new_feature)
        
    return new_features

In [40]:
train_set = rebuild_features(train_set)
test_set = rebuild_features(test_set) 

In [41]:
class MaxEntropy(object):
    
    def init_params(self,X,Y):
        self.X = X
        self.Y = set()
        
        self.cal_Pxy_Px(X,Y)
        
        self.N = len(X)
        self.n = len(self.Pxy) # (x,y) pair的对数
        self.M = 1000.0 #学习率
        
        self.build_dict()
        self.cal_EPxy() 
    
    
    def cal_Pxy_Px(self,X,Y):
        '''
        Count empirical appearance of each feature and feature-value pair
        '''
        self.Pxy = defaultdict(int) #对应init params里的n
        self.Px  = defaultdict(int)
        
        for i in xrange(len(X)):
            x_,y = X[i],Y[i]
            self.Y.add(y)
            
            for x in x_:
                self.Pxy[(x,y)] += 1
                self.Px[x] += 1
        
    def build_dict(self):
        self.id2xy = {}
        self.xy2id = {}
        
        for i, (x,y) in enumerate(self.Pxy):
            self.id2xy[i] = (x,y)
            self.xy2id[(x,y)] = i
    
    def cal_EPxy(self):
        '''
        计算P.82 最下面f(x,y) 关于经验分布p~(X,Y)的期望
        '''
        self.EPxy = defaultdict(float)
        
        for id in xrange(self.n):
            (x,y) = self.id2xy[id]
            self.EPxy[id] = float(self.Pxy[(x,y)]) / float(self.N) #why id here 
    
    
    def fxy(self,x,y):
        return (x,y) in self.xy2id
    
    def cal_pyx(self,X,y):
        '''
        Helper for cal_probability()
        '''
        result = 0.0
        for x in X:
            if self.fxy(x,y):
                id = self.xy2id[(x,y)]
                result += self.w[id]  #对应权值向量w的一个entry
        
        return (np.exp(result),y) 
    
    def cal_probability(self,X):
        '''
        计算P.85 equation (6.22) 
        '''
        Pyxs = [(self.cal_pyx(X, y)) for y in self.Y]
        Z = np.sum([prob for prob,y in Pyxs]) 
        return [(prob/Z, y) for prob,y in Pyxs]        
        
    def cal_EPx(self):
        '''
        P.83最上面的期望， f(x,y) 关于P(Y|X) 和经验分布p~(X)的期望
        '''
        self.EPx = [0.0 for i in range(self.n)]
    
        for i,X in enumerate(self.X):
            Pyxs = self.cal_probability(X) 
            
            for x in X:
                for Pyx, y in Pyxs:
                    if self.fxy(x,y):
                        id = self.xy2id[(x,y)]
                        
                        self.EPx[id] += Pyx*(1.0/self.N) ###why 1/length
                        #self.EPx[id] += Pyx*(self.Px[x]/self.n)
        
    def train(self,X,Y):
        '''
        Train MaxEn model
        '''
        self.init_params(X,Y)
        self.w = [0.0 for i in range(self.n)]
        self.w = np.array(self.w)
        
        #with this model while using 25 iterations it takes around 1 hour
        #can't get result when using large iteration number (like 1000 times) 
        max_iteration = 25 
        
        for iterate in xrange(max_iteration):
            sigmas = []
            self.cal_EPx()
            
            for i in xrange(self.n):
                sigma = 1 / self.M * np.log(self.EPxy[i]/self.EPx[i])
                sigmas.append(sigma)
            
            sigmas = np.array(sigmas)
            self.w += sigmas
            #print sigmas
    
    def predict(self,testset):
        results = []
        for test in testset:
            result = self.cal_probability(test)
            
            results.append((max(result, key=lambda x: x[0])[1]))
        return results

In [42]:
met = MaxEntropy()
time_1 = time.time()
met.train(train_set, train_labels)
time_2 = time.time()

time_3 = time.time()
print 'training cost ', time_2 - time_1, ' second', '\n'

print 'Start predicting'
test_predict = met.predict(test_set)
time_4 = time.time()
print 'predicting cost ', time_4 - time_3, ' second', '\n'

score = accuracy_score(test_labels, test_predict)
print score

training cost  4058.19630003  second 

Start predicting
predicting cost  33.9070739746  second 

0.808484848485


In [43]:
max(met.EPxy.values()) 

0.1091044776119403

In [45]:
met.w

array([ 0.0040996 , -0.00407161,  0.02080868, ...,  0.02773535,
       -0.003552  ,  0.02241716])